In [1]:
# Google Colab-only setup. No need to run this cell in other environments.
use_colab = False

if use_colab:
    # Mount my Google Drive root folder
    from google.colab import drive
    drive.mount('/content/drive')

    # cd to bayesian-dl-experiments directory
    %cd 'drive/My Drive/Colab Notebooks/bayesian-dl-experiments'
    !ls

## Experiment Setup

In [2]:
import torch
import numpy as np

# IPython reloading magic
%load_ext autoreload
%autoreload 2

# Random seeds
# Based on https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(682)
np.random.seed(682)

# torch.device / CUDA Setup
use_cuda = True

if use_cuda and torch.cuda.is_available():
    torch_device = torch.device('cuda')
    torch.backends.cudnn.deterministic = True
    # Note: https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
    torch.backends.cudnn.benchmark = False
else:
    torch_device = torch.device('cpu')

# Dataset to use
dataset_name = 'yacht'

# Training set size
dataset_train_size = 0.8

# L2 regularization strength
reg_strength = 0.01

# Epochs
n_epochs = 10

# Number of different data splits to try
n_splits = 2

# Data batch sizes
n_training_batch = 10

# Number of test predictions (for each data point)
n_predictions = 10000

## Prepare data

### Get the data as a torch Dataset object

In [3]:
from torch.utils.data import random_split, DataLoader
from ronald_bdl import datasets

if dataset_name == 'MNIST':
    dataset = datasets.MNIST(root_dir='./datasets_files', transform=None, download=True)
else:
    dataset = datasets.UCIDatasets(dataset_name, root_dir='./datasets_files', transform=None, download=True)

train_size = int(dataset_train_size * len(dataset))
test_size = len(dataset) - train_size
    
# Print the size of the dataset
print("dataset size = " + str(dataset.data.shape))
print("training set size = " + str(train_size))
print("testing set size = " + str(test_size))

Using downloaded and verified file: ./datasets_files/yacht/yacht_hydrodynamics.data
dataset size = torch.Size([308, 7])
training set size = 246
testing set size = 62


## Define network

In [4]:
from ronald_bdl import models

network = models.FCNetMCDropout(
    input_dim=len(dataset.features), 
    output_dim=len(dataset.targets),
    hidden_dim=100,
    n_hidden=2,
    dropout_rate=0.01,
)

# Send the whole model to the selected torch.device
network.to(torch_device)

# Print the network structure
print(network)

FCNetMCDropout(
  (input): Linear(in_features=6, out_features=100, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
  )
  (output): Linear(in_features=100, out_features=1, bias=True)
)


## Train the network

### Setup optimizer

In [5]:
from torch import nn, optim

# Model to train mode
network.train()

# Mean Squared Error for loss function to minimize
objective = nn.MSELoss()

# Adam optimizer
# https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam
# NOTE: Need to set L2 regularization from here
optimizer = optim.Adam(
    network.parameters(),
    lr=0.01, 
    weight_decay=reg_strength # L2 regularization
)

rmse_non_mc, rmse_mc, test_lls_mc = [], [], []

### Train/test the model

In [6]:
for s in range(n_splits):

    # Prepare new train-test split
    train, test = random_split(dataset, lengths=[train_size, test_size])
    train_loader = DataLoader(train, batch_size=n_training_batch)
    
    # Training
    for epoch in range(n_epochs): # loop over the dataset multiple times

        print("Running split " + str(s) + ", epoch " + str(epoch))

        for i, data in enumerate(train_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, targets = data

            # Store the batch to torch_device's memory
            inputs = inputs.to(torch_device)
            targets = targets.to(torch_device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = network(inputs)

            loss = objective(outputs, targets)
            loss.backward()

            optimizer.step()
            
    # Testing

    # Model to eval mode
    network.eval()

    # Get the test data
    inputs, targets = test.dataset[test.indices]

    # Store the batch to torch_device's memory
    inputs = inputs.to(torch_device)
    targets = targets.to(torch_device)

    predictions, mean, var, metrics = network.mc_predict(inputs, n_predictions,
                                                         y_test=targets, reg_strength=reg_strength)

    print()
    print("Running split " + str(s) + " test:")
    print("Mean = " + str(mean))
    print("Variance = " + str(var))

    # Print and store additional metrics
    if len(metrics) > 0:
        for key, value in metrics.items():
            print(str(key) + " = " + str(value))

            if key == 'rmse_mc': rmse_mc.append(value)
            elif key == 'rmse_non_mc': rmse_non_mc.append(value)
            elif key == 'test_ll_mc': test_lls_mc.append(value)
    print()

Running split 0, epoch 0
Running split 0, epoch 1
Running split 0, epoch 2
Running split 0, epoch 3
Running split 0, epoch 4
Running split 0, epoch 5
Running split 0, epoch 6
Running split 0, epoch 7
Running split 0, epoch 8
Running split 0, epoch 9

Running split 0 test:
Mean = tensor([[ 93.8531],
        [ 89.6675],
        [ 67.5957],
        [ 62.2736],
        [ 40.5143],
        [ 45.0080],
        [ 81.6247],
        [ 82.1640],
        [102.0260],
        [ 61.2794],
        [ 70.5819],
        [100.4111],
        [ 63.1331],
        [ 70.6257],
        [115.0932],
        [126.1112],
        [ 78.4996],
        [ 97.8473],
        [ 90.9057],
        [ 58.2732],
        [ 93.8544],
        [ 59.0250],
        [ 36.0878],
        [ 72.8479],
        [107.6530],
        [100.9108],
        [109.9260],
        [ 72.1726],
        [ 64.3754],
        [ 72.3660],
        [ 86.0217],
        [ 65.4583],
        [ 75.4726],
        [ 81.6691],
        [ 50.7310],
        [ 75.2734],


### Print statistics

In [10]:
# Copied from DropoutUncertaintyExps repo
print('non-MC RMSE %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(rmse_non_mc), np.std(rmse_non_mc), np.std(rmse_non_mc)/np.sqrt(n_splits),
        np.percentile(rmse_non_mc, 50), np.percentile(rmse_non_mc, 25), np.percentile(rmse_non_mc, 75)))

print('MC RMSE %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(rmse_mc), np.std(rmse_mc), np.std(rmse_mc)/np.sqrt(n_splits),
        np.percentile(rmse_mc, 50), np.percentile(rmse_mc, 25), np.percentile(rmse_mc, 75)))

print('MC Test Log-likelihood %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(test_lls_mc), np.std(test_lls_mc), np.std(test_lls_mc)/np.sqrt(n_splits), 
        np.percentile(test_lls_mc, 50), np.percentile(test_lls_mc, 25), np.percentile(test_lls_mc, 75)))

non-MC RMSE 7.631426 +- 2.625468 (stddev) +- 1.856486 (std error), median 7.631426 25p 6.318692 75p 8.944160 

MC RMSE 7.684747 +- 2.574000 (stddev) +- 1.820093 (std error), median 7.684747 25p 6.397747 75p 8.971747 

MC Test Log-likelihood -3.552364 +- 0.194530 (stddev) +- 0.137554 (std error), median -3.552364 25p -3.649629 75p -3.455099 

